In [39]:
import numpy
import matplotlib # for exporting graphs
matplotlib.use('Agg') # for exporting graphs
import matplotlib.pyplot as pyplot
from tensorflow import keras
from TrainingDataGeneratorV3 import training_data_generator

fig_size = (2*6.4, 2*4.8)

In [2]:
train_data, test_data, time_steps, space_steps = training_data_generator(
    num_samples=15, # FOR A TRAIN/TEST SPLIT OF 0.2 5 IS THE MIN FOR 1 TEST SAMPLE
    num_potentials=5,
    test_set_rnd_seed=0)

Building training data, 15 samples:
>>> Generating random initial wavefunctions
>>> Success 0.003092050552368164
>>> Generating random potentials
(5, 1024)
>>> Success 0.0009410381317138672
>>> Solving pseudo-spectral ODE...
    0 of 15, Batch time: 3.814697265625e-06
    10 of 15, Batch time: 28.692018270492554
    0 of 15, Batch time: 13.445857048034668
    10 of 15, Batch time: 27.650409936904907
    0 of 15, Batch time: 13.604524850845337
    10 of 15, Batch time: 28.828989028930664
    0 of 15, Batch time: 14.23098611831665
    10 of 15, Batch time: 27.590330839157104
    0 of 15, Batch time: 13.448508262634277
    10 of 15, Batch time: 27.11696171760559
>>> Success 208.320650100708
Total elapse time: 208.76601433753967


In [3]:
print(train_data['input'].shape)
#train_data_input = train_data['input'].reshape(500, train_data['input'].shape[0]*train_data['input'].shape[1]*1024)
train_data_input = train_data['input'].reshape(train_data['input'].shape[0]*train_data['input'].shape[1], 500, 1024)
print(train_data_input.shape)

print(train_data['output'].shape)
train_data_output = train_data['output'].reshape(train_data['output'].shape[0]*train_data['output'].shape[1], 500, 1024)
print(train_data_output.shape)

(5, 12, 500, 1024)
(60, 500, 1024)
(5, 12, 500, 1024)
(60, 500, 1024)


In [4]:
num_epochs = 50

# WHAT IS THE FORM OF THE INPUT? IS IT COMPLEX?
# IF YOU ARE USING COMPLEX VALUES, YOU NEED TO CONSIDER THINGS LIKE THE ACTIVATION FUNCTION
# AND HOW IT HANDLES THINGS (FOR EXAMPLE GREATER/LESS THAN COMPARISON AREN'T DEFINED FOR COMPLEX NUMBERS)
# IF IT IS COMPLEX, HOW IS THAT BEING HANDLED? WHAT IS GOING ON WITH YOUR METHOD UNDER THE HOOD?
# NEED TO UNDERSTAND HOW THINGS ARE BEING HANDLED BY KERAS.

# IF IT IS COMPLEX NUMBERS BEING FED INTO THE NETWORK, YOU ARE GETTING A REAL NUMBER BACK, OR IT'S GIVING
# YOU A COMPLEX NUMBER BUT ALL OF YOUR TRAINING DATA IS REAL


# create input nodes
nn_input = keras.Input(shape=(500, 1024))

# create hidden layers, connect graph nodes (construct graph structure)
nn_output = keras.layers.Dense(units=2048, activation='elu')(nn_input)
nn_output = keras.layers.Dense(units=2048, activation='elu')(nn_output)
nn_output = keras.layers.Dense(1024)(nn_output)

# create a model from the graph
nn_model = keras.Model(inputs=nn_input, outputs=nn_output)

# complile model parameters
nn_model.compile(optimizer='Adam', loss='MSE')

# fit the model
nn_hist = nn_model.fit(x=train_data_input,
                       y=train_data_output,
                       batch_size=5,
                       validation_split=0.2,
                       epochs=num_epochs,
                       verbose=1
                      )

Epoch 1/50
10/10 [==============================] - 5s 510ms/step - loss: 0.0497 - val_loss: 0.0533
Epoch 2/50
10/10 [==============================] - 5s 448ms/step - loss: 0.0285 - val_loss: 0.0310
Epoch 3/50
10/10 [==============================] - 5s 451ms/step - loss: 0.0229 - val_loss: 0.0226
Epoch 4/50
10/10 [==============================] - 4s 448ms/step - loss: 0.0200 - val_loss: 0.0183
Epoch 5/50
10/10 [==============================] - 4s 446ms/step - loss: 0.0171 - val_loss: 0.0158
Epoch 6/50
10/10 [==============================] - 5s 453ms/step - loss: 0.0173 - val_loss: 0.0162
Epoch 7/50
10/10 [==============================] - 5s 453ms/step - loss: 0.0159 - val_loss: 0.0151
Epoch 8/50
10/10 [==============================] - 4s 446ms/step - loss: 0.0153 - val_loss: 0.0153
Epoch 9/50
10/10 [==============================] - 5s 455ms/step - loss: 0.0158 - val_loss: 0.0142
Epoch 10/50
10/10 [==============================] - 5s 455ms/step - loss: 0.0151 - val_loss: 0.0138

In [5]:
nn_hist.history.keys()

dict_keys(['loss', 'val_loss'])

In [44]:
xaxis = range(1, num_epochs + 1)

pyplot.figure(figsize=fig_size)
pyplot.plot(xaxis, nn_hist.history['loss'], label='Training Loss')
pyplot.plot(xaxis, nn_hist.history['val_loss'], label='Validation Loss')
pyplot.title('Training metrics', fontsize=20)
pyplot.xlabel('Epochs', fontsize=20)
pyplot.legend(fontsize=20)
pyplot.savefig('training-metrics') # for exporting graphs
pyplot.close() # for exporting graphs
#pyplot.show()

In [7]:
print(test_data['input'].shape)
test_data_input = test_data['input'].reshape(test_data['input'].shape[0]*test_data['input'].shape[1], 500, 1024)
print(test_data_input.shape)

(5, 3, 500, 1024)
(15, 500, 1024)


In [8]:
nn_predict = nn_model.predict(x=test_data_input,
                              verbose=True)
#nn_predict = nn_model.predict(x=numpy.array([test_data['input'][0,0]]),
#                              verbose=True)

1/1 [==============================] - 0s 330ms/step


In [9]:
print(nn_predict.shape)
print(test_data['output'].shape)

(15, 500, 1024)
(5, 1024)


In [45]:
samples_per_potential = int(nn_predict.shape[0] / test_data['output'].shape[0])
print(samples_per_potential)

for p in range(test_data['output'].shape[0]): # potential
    index = samples_per_potential*p
    pyplot.figure(figsize=fig_size)
    for s in range(index, index+samples_per_potential+1): # sample
        for i in range(nn_predict.shape[1]): # time evolution of sample
            pyplot.plot(space_steps, nn_predict[p,i], 'c')
    
    pyplot.plot(space_steps, test_data['output'][p], 'r', label='Truth')
    pyplot.title(f'Potential {p+1}', fontsize=20)
    pyplot.xlabel('Position (no units)', fontsize=20)
    pyplot.ylabel('Amplitude (no units)', fontsize=20)
    pyplot.legend(fontsize=20)
    pyplot.savefig(f'potential{p+1}') # for exporting graphs
    pyplot.close() # for exporting graphs
    #pyplot.show()



3
